# GISTDA Wildfire Machine Learning Training

## Import Library Packages

#### Core Libraries
- **numpy**: A powerful library for numerical computing, providing support for arrays, matrices, and a wide range of mathematical operations.
- **pandas**: A versatile library for data manipulation and analysis, offering easy-to-use data structures like `DataFrame` for handling tabular data.

#### Visualization Libraries
- **matplotlib.pyplot**: A plotting library for creating static, animated, and interactive visualizations in Python.
- **seaborn**: Built on top of Matplotlib, this library provides a high-level interface for drawing attractive and informative statistical graphics.

#### Geospatial Libraries
- **rasterio**: A library for reading and writing geospatial raster data, commonly used for remote sensing and GIS applications.
  - **rasterio.windows.Window**: A tool to define a specific rectangular region (window) within a raster dataset for operations.

#### Logging
- **logging**: A built-in Python module to record log messages, useful for debugging and monitoring program execution.

#### File Handling
- **os**: A module providing tools to interact with the operating system, such as reading/writing files and directory management.
- **pickle**: A module for serializing and deserializing Python objects for storage or transmission.

#### Data Preprocessing
- **sklearn.preprocessing.MinMaxScaler**: A preprocessing tool from scikit-learn that scales and normalizes data within a specified range, often [0, 1].

#### Display Utilities
- **IPython.display.display**: A utility to control the display of outputs in Jupyter notebooks.
- **IPython.display.Markdown**: Enables the rendering of Markdown text in a Jupyter notebook.

#### Deep Learning Libraries
- **tensorflow**: A popular open-source framework for machine learning and deep learning. It supports building and training neural networks efficiently.
  - **tensorflow.keras.models.Sequential**: A simple model-building API for stacking layers sequentially.
  - **tensorflow.keras.layers.Dense**: A fully connected neural network layer.
  - **tensorflow.keras.layers.Dropout**: A regularization technique to prevent overfitting by randomly dropping neurons during training.
  - **tensorflow.keras.callbacks.EarlyStopping**: Stops training when a monitored metric stops improving.
  - **tensorflow.keras.callbacks.ReduceLROnPlateau**: Adjusts the learning rate when a metric stops improving.

#### Evaluation Metrics
- **sklearn.metrics.classification_report**: Generates a detailed report of classification metrics, including precision, recall, and F1 score.
- **sklearn.metrics.confusion_matrix**: Computes a confusion matrix to evaluate classification accuracy.
- **sklearn.metrics.roc_curve**: Computes Receiver Operating Characteristic (ROC) curve metrics.
- **sklearn.metrics.auc**: Computes the Area Under the Curve (AUC) for a ROC curve.

#### Typing
- **typing**: Provides tools for type annotations to improve code clarity and static analysis.
  - **Tuple, Dict, Any, List**: Common data types used in type hinting.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio
import logging
import os
import pickle
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display, Markdown
from rasterio.windows import Window
from typing import Tuple, Dict, Any, List
from dask import delayed, compute

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

## Import Raster GeoTIFF files

This code reads large raster files in manageable chunks, processes each chunk, and stores the data in a consolidated DataFrame.

1. **Folder Path and Chunk Size**: Sets the path to the folder containing raster files (`raster_train_file_path`) and defines the chunk size (`CHUNK_SIZE`).

2. **`read_raster_in_chunks` Function**:
   - Reads a raster file chunk-by-chunk.
   - For each chunk, it extracts data from all bands, along with the pixel coordinates.
   - Stores each chunk's data in a temporary DataFrame, which is added to a list.

3. **Task Creation**:
   - Creates a list of tasks using Dask's `@delayed` to process each raster file in parallel.

4. **Compute and Combine**:
   - Computes each task to get individual DataFrames for each file, then combines them into a final DataFrame (`final_df`).

5. **Debug Outputs**:
   - Prints the shape, columns, and a sample of the final DataFrame for inspection.

In [2]:
# Define the folder containing the raster files
raster_train_file_path = r'Raster_Train'

# Parameters for chunk size
CHUNK_SIZE = 1024

@delayed
def read_raster_in_chunks(raster_path, file, root):
    with rasterio.open(raster_path) as src:
        height, width = src.height, src.width
        num_bands = src.count
        band_names = [f'B{str(i).zfill(2)}' for i in range(1, num_bands + 1)]
        chunk_dfs = []
        
        # Loop over the raster in chunks
        for row in range(0, height, CHUNK_SIZE):
            for col in range(0, width, CHUNK_SIZE):
                window = Window(col_off=col, row_off=row, 
                              width=min(CHUNK_SIZE, width - col),
                              height=min(CHUNK_SIZE, height - row))
                
                # Read all bands at once
                data = src.read(window=window)
                
                # Check if chunk contains any data
                if np.any(data):
                    rows, cols = data[0].shape
                    
                    # Create base DataFrame with coordinates
                    row_coords, col_coords = np.meshgrid(
                        np.arange(row, row + rows),
                        np.arange(col, col + cols),
                        indexing="ij"
                    )
                    
                    chunk_df = pd.DataFrame({
                        'raster_file': file,
                        'subfolder': os.path.basename(root),
                        'x': row_coords.flatten(),
                        'y': col_coords.flatten()
                    })
                    
                    # Add each band's data
                    for band_idx, band_name in enumerate(band_names, 1):
                        chunk_df[band_name] = data[band_idx-1].flatten()
                    
                    chunk_dfs.append(chunk_df)
        
        return pd.concat(chunk_dfs, ignore_index=True) if chunk_dfs else pd.DataFrame()

# Create list of tasks
dask_dfs = [
    read_raster_in_chunks(os.path.join(root, file), file, root)
    for root, dirs, files in os.walk(raster_train_file_path)
    for file in files if file.endswith('.tif')
]

# Compute all tasks
dataframes = compute(*dask_dfs)

# Combine all DataFrames
final_df = pd.concat(dataframes, ignore_index=True)

# Debug prints
print("DataFrame shape:", final_df.shape)
print("\nDataFrame columns:", final_df.columns.tolist())
print("\nSample of data:")
print(final_df.head())

DataFrame shape: (28196864, 20)

DataFrame columns: ['raster_file', 'subfolder', 'x', 'y', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'B16']

Sample of data:
                                  raster_file     subfolder  x  y     B01  \
0  T46QHM_20230316T040541_chunk_8192_4096.tif  Raster_Train  0  0  1561.0   
1  T46QHM_20230316T040541_chunk_8192_4096.tif  Raster_Train  0  1  1561.0   
2  T46QHM_20230316T040541_chunk_8192_4096.tif  Raster_Train  0  2  1564.0   
3  T46QHM_20230316T040541_chunk_8192_4096.tif  Raster_Train  0  3  1564.0   
4  T46QHM_20230316T040541_chunk_8192_4096.tif  Raster_Train  0  4  1561.0   

      B02     B03     B04     B05     B06     B07     B08     B09     B10  \
0  1608.0  1725.0  1857.0  2184.0  2348.0  2510.0  2700.0  2712.0  2737.0   
1  1614.0  1782.0  1912.0  2184.0  2348.0  2510.0  2686.0  2712.0  2737.0   
2  1554.0  1734.0  1888.0  2201.0  2399.0  2614.0  2588.0  2790.0  2778.0   
3  1579.0

## Exploratory Data Analysis (EDA) & Feature Engineering

This code performs several operations on a DataFrame (`final_df`), converting it to a Dask DataFrame for distributed processing, renaming columns, and dropping unnecessary columns:

1. **Convert to Dask DataFrame**:
   - Converts the existing Pandas DataFrame (`final_df`) into a Dask DataFrame (`ddf`) to enable parallel, distributed processing.

2. **Rename Columns**:
   - Defines a list of new column names for Sentinel-2 bands and other data (`new_col_names`).
   - Renames `ddf` columns using this list to make them more descriptive.

3. **Drop Unneeded Columns**:
   - Drops columns such as `raster_file`, `subfolder`, `x`, `y`, and `dNBR`, keeping only essential information in the DataFrame (`df`).

4. **Display DataFrame**:
   - Uses `display()` to view the DataFrame when needed, triggering Dask's computation.

In [3]:
# Convert to Dask DataFrame
ddf = pd.DataFrame(final_df)  # Adjust npartitions as needed

# Rename Sentinel-2 Bands columns
new_col_names = ['raster_file', 'subfolder', 'x', 'y', 'Band_1', 'Band_2', 'Band_3', 'Band_4', 'Band_5', 
                 'Band_6', 'Band_7', 'Band_8', 'Band_8A', 'Band_9', 'Band_11', 'Band_12', 'dNBR',
                 'NDVI', 'NDWI', 'Burn_Label']

# Rename columns
final_df.columns = new_col_names

# Drop columns using Dask method
df = final_df.drop(columns=['raster_file', 'subfolder', 'x', 'y', 'dNBR'])
display(df)  # Compute only when you need to display or save results

,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7,Band_8,Band_8A,Band_9,Band_11,Band_12,NDVI,NDWI,Burn_Label
0,1561.0,1608.0,1725.0,1857.0,2184.0,2348.0,2510.0,2700.0,2712.0,2737.0,3146.0,2620.0,0.184990,-0.220339,0.0
1,1561.0,1614.0,1782.0,1912.0,2184.0,2348.0,2510.0,2686.0,2712.0,2737.0,3146.0,2620.0,0.168334,-0.202328,0.0
2,1564.0,1554.0,1734.0,1888.0,2201.0,2399.0,2614.0,2588.0,2790.0,2778.0,3122.0,2541.0,0.156390,-0.197594,0.0
3,1564.0,1579.0,1752.0,1929.0,2201.0,2399.0,2614.0,2672.0,2790.0,2778.0,3122.0,2541.0,0.161487,-0.207957,0.0
4,1561.0,1626.0,1786.0,1980.0,2238.0,2539.0,2680.0,2714.0,2933.0,2778.0,3228.0,2632.0,0.156370,-0.206222,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28196859,1407.0,1503.0,1665.0,1660.0,2177.0,2938.0,3265.0,3397.0,3601.0,3620.0,3258.0,2321.0,0.343484,-0.342157,0.0
28196860,1402.0,1481.0,1693.0,1692.0,2287.0,3069.0,3370.0,3642.0,3697.0,3620.0,3344.0,2343.0,0.365579,-0.365323,0.0
28196861,1402.0,1512.0,1736.0,1769.0,2287.0,3069.0,3370.0,3576.0,3697.0,3620.0,3344.0,2343.0,0.338073,-0.346386,0.0
28196862,1391.0,1538.0,1768.0,1828.0,2325.0,3000.0,3323.0,3521.0,3692.0,3661.0,3326.0,2315.0,0.316508,-0.331443,0.0


### Check Burn Class

This code checks and displays the counts of burn records in the DataFrame:

1. **Count Burn Labels**:
   - Counts occurrences in the `Burn_Label` column to determine the number of "Burn" and "Unburn" records.
   - Renames the labels: `1` to "Burn" and `0` to "Unburn" for readability.

2. **Display Counts**:
   - Prints the resulting counts to show the distribution of burn and unburned areas.

In [4]:
# Check Burn Records
burn_counts = df['Burn_Label'].value_counts().rename(index={1: 'Burn', 0: 'Unburn'})

# Display the counts with labels
print(burn_counts)

Burn_Label
Unburn    25742453
Burn       2454411
Name: count, dtype: int64


### Downsampling

This code performs downsampling to balance the dataset by reducing the number of "Unburn" records to match the count of "Burn" records:

1. **Get Burn Count**:
   - Retrieves the count of "Burn" records from `burn_counts`.

2. **Sample Unburned Records**:
   - Selects a random sample of "Unburn" records, equal in size to the number of "Burn" records, using a fixed `random_state` for reproducibility.

3. **Combine Burn and Downsampled Unburn Records**:
   - Combines all "Burn" records with the downsampled "Unburn" sample into a new DataFrame (`downsampled_df`).

4. **Check New Burn Record Counts**:
   - Counts and displays the "Burn" and "Unburn" records in `downsampled_df` to verify balance.

In [5]:
burn_count = burn_counts['Burn']
unburn_sample = df[df['Burn_Label'] == 0].sample(n=burn_count, random_state=42)

downsampled_df = pd.concat([df[df['Burn_Label'] == 1], unburn_sample])

# Check Burn Records
burn_counts = downsampled_df['Burn_Label'].value_counts().rename(index={1: 'Burn', 0: 'Unburn'})

# Display the counts with labels
print(burn_counts)

Burn_Label
Burn      2454411
Unburn    2454411
Name: count, dtype: int64


### Remove infinite values

This code handles the presence of infinite values and missing data in the `downsampled_df` DataFrame:

1. **Replace Infinite Values**:
   - Replaces both positive and negative infinite values (`np.inf` and `-np.inf`) with `NaN` using `replace()`. This ensures that infinite values do not interfere with further processing.

2. **Drop Rows with Missing Values**:
   - Removes any rows containing `NaN` values using `dropna()`, ensuring the DataFrame only contains valid data.

3. **Display the DataFrame**:
   - Displays the cleaned DataFrame (`downsampled_df`) for inspection.

In [6]:
# Replacing infinite with nan 
downsampled_df.replace([np.inf, -np.inf], np.nan, inplace=True) 
  
# Dropping all the rows with nan values 
downsampled_df.dropna(inplace=True)

# Printing df 
display(downsampled_df)

,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7,Band_8,Band_8A,Band_9,Band_11,Band_12,NDVI,NDWI,Burn_Label
15,1535.0,1608.0,1720.0,1900.0,2209.0,2374.0,2526.0,2440.0,2749.0,2688.0,3252.0,2792.0,0.124424,-0.173077,1.0
16,1533.0,1583.0,1688.0,1866.0,2075.0,2246.0,2370.0,2382.0,2567.0,2688.0,3106.0,2673.0,0.121469,-0.170516,1.0
17,1533.0,1582.0,1662.0,1879.0,2075.0,2246.0,2370.0,2428.0,2567.0,2688.0,3106.0,2673.0,0.127467,-0.187286,1.0
20,1530.0,1522.0,1643.0,1735.0,1959.0,2092.0,2167.0,2198.0,2306.0,2404.0,2902.0,2643.0,0.117722,-0.144494,1.0
21,1530.0,1570.0,1613.0,1774.0,1959.0,2092.0,2167.0,2016.0,2306.0,2404.0,2902.0,2643.0,0.063852,-0.111050,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029686,1699.0,1669.0,1860.0,1910.0,2308.0,2701.0,2877.0,2954.0,3217.0,3334.0,3475.0,2515.0,0.214638,-0.227254,0.0
4749561,1610.0,1737.0,1879.0,2218.0,2454.0,2656.0,2817.0,2920.0,3084.0,2787.0,3859.0,3257.0,0.136629,-0.216920,0.0
5034484,1513.0,1534.0,1705.0,1978.0,2249.0,2409.0,2651.0,2744.0,2953.0,2971.0,3958.0,3189.0,0.162219,-0.233536,0.0
13797249,1114.0,1161.0,1350.0,1216.0,1703.0,3287.0,3855.0,4106.0,4187.0,4168.0,2405.0,1605.0,0.543029,-0.505132,0.0


### Seperate Burn_Label from DataFrame

This code separates the `Burn_Label` from the main DataFrame and ensures the label is in the correct format:

1. **Separate Burn Label**:
   - Extracts the `Burn_Label` column from `downsampled_df` into a new DataFrame (`burn_label`).

2. **Remove Burn Label from Main DataFrame**:
   - Drops the `Burn_Label` column from `downsampled_df` to ensure only the feature data remains.

3. **Convert Burn Label to Integer**:
   - Changes the data type of the `burn_label` DataFrame to `int32` to ensure consistent and efficient processing.

4. **Display Burn Label**:
   - Displays the modified `burn_label` DataFrame to verify the changes.

In [7]:
# Seperate Burn_Label from DataFrame
burn_label = downsampled_df[['Burn_Label']]

# Drop Label from DataFrame
downsampled_df = downsampled_df.drop(columns=['Burn_Label'])

# Change type of Label to Integer Format
burn_label = burn_label.astype('int32')
display(burn_label)

,Burn_Label
15,1
16,1
17,1
20,1
21,1
...,...
2029686,0
4749561,0
5034484,0
13797249,0


### Normalization Data with MinMax Scaler

This code normalizes the feature data and saves the normalization model, while also combining it with the `Burn_Label`:

1. **List Columns**:
   - Creates a list of column names from `downsampled_df` to keep track of the column order after normalization (`cols_norm`).

2. **Normalize the Data**:
   - Imports `MinMaxScaler` from `sklearn` and fits it to the data in `downsampled_df`, which scales the values between 0 and 1.

3. **Save the Scaler**:
   - Saves the fitted `MinMaxScaler` model to a specified path (`MinMax_Scaler.pkl`) for later use.

4. **Apply Normalization**:
   - Normalizes the data by applying the `scaler` to `downsampled_df`, then converts the result back into a DataFrame (`df_norm`) with the original column names.

5. **Check Shape**:
   - Prints the shape of `df_norm` to confirm the normalization was applied correctly.

6. **Concatenate with Burn Label**:
   - Combines the normalized feature data (`df_norm`) with the `burn_label` DataFrame, aligning them by their indices and ensuring the result is a complete dataset.

7. **Display the DataFrame**:
   - Displays the final DataFrame (`df_norm`), which now includes both the normalized features and the `Burn_Label`.


In [8]:
# Reassign the dataframe with a list of the columns
cols_norm = downsampled_df.columns.tolist()

# Import Normalize technique
scaler = MinMaxScaler()

# Normalize data
scaler.fit(downsampled_df)

# Save the scaler
scaler_save_path = r'Export Model'
save_path = os.path.join(scaler_save_path, 'MinMax_Scaler_DL.pkl')
os.makedirs(os.path.dirname(save_path), exist_ok=True)
with open(save_path, 'wb') as f:
    pickle.dump(scaler, f)

# Normalize Data
df_norm = scaler.transform(downsampled_df)
df_norm = pd.DataFrame(df_norm, columns=cols_norm)

# Check df_norm shape after normalization
print("Shape of df_norm after normalization:", df_norm.shape)

# Concatenate df_norm with burn_label
df_norm = pd.concat([df_norm.reset_index(drop=True), burn_label.reset_index(drop=True)], axis=1, sort=False)
display(df_norm)

Shape of df_norm after normalization: (4908822, 14)


,Band_1,Band_2,Band_3,Band_4,Band_5,Band_6,Band_7,Band_8,Band_8A,Band_9,Band_11,Band_12,NDVI,NDWI,Burn_Label
0,0.324940,0.187784,0.172900,0.187760,0.210077,0.194398,0.201447,0.149154,0.206588,0.342417,0.189673,0.113378,0.408953,0.500569,1
1,0.324141,0.179662,0.164228,0.180328,0.181469,0.170495,0.175793,0.141722,0.181505,0.342417,0.177108,0.105695,0.405882,0.503316,1
2,0.324141,0.179337,0.157182,0.183169,0.181469,0.170495,0.175793,0.147617,0.181505,0.342417,0.177108,0.105695,0.412115,0.485325,1
3,0.322942,0.159844,0.152033,0.151694,0.156704,0.141737,0.142411,0.118145,0.145535,0.291502,0.159552,0.103758,0.401988,0.531233,1
4,0.322942,0.175439,0.143902,0.160219,0.156704,0.141737,0.142411,0.094823,0.145535,0.291502,0.159552,0.103758,0.346003,0.567111,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4908817,0.390488,0.207602,0.210840,0.189945,0.231213,0.255462,0.259168,0.215018,0.271086,0.458229,0.208864,0.095493,0.502709,0.442447,0
4908818,0.354916,0.229695,0.215989,0.257268,0.262383,0.247059,0.249301,0.210661,0.252756,0.360165,0.241910,0.143401,0.421637,0.453533,0
4908819,0.316147,0.163743,0.168835,0.204809,0.218617,0.200934,0.222003,0.188109,0.234702,0.393152,0.250430,0.139011,0.448232,0.435708,0
4908820,0.156675,0.042560,0.072629,0.038251,0.102050,0.364893,0.419997,0.362635,0.404768,0.607745,0.116781,0.036738,0.843992,0.144339,0


## Deep Learning Hyperparameters Analysis & Training

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Ensure df_norm is already defined and preprocessed
# Define the features and target
X = df_norm.drop(columns=['Burn_Label'])  # Features
y = df_norm['Burn_Label']  # Target

# Convert labels to one-hot encoding
y_onehot = tf.keras.utils.to_categorical(y, num_classes=y.nunique())

# Function to build the model
def build_model(hidden_layers=1):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(X.shape[1],)))
    for _ in range(hidden_layers):
        model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(y_onehot.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Hyperparameter Grid
hidden_layers_options = [1, 2, 3]
epochs_options = [50, 100]
batch_size_options = [16, 32]

# Grid Search for Best Hyperparameters
best_params = None
best_score = 0

for layers in hidden_layers_options:
    for epochs in epochs_options:
        for batch_size in batch_size_options:
            # Perform k-fold cross-validation
            kfold = KFold(n_splits=10, shuffle=True, random_state=42)
            scores = []
            for train_idx, val_idx in kfold.split(X):
                model = build_model(hidden_layers=layers)
                model.fit(X.iloc[train_idx], y_onehot[train_idx], epochs=epochs, batch_size=batch_size, verbose=0)
                score = model.evaluate(X.iloc[val_idx], y_onehot[val_idx], verbose=0)[1]
                scores.append(score)
            avg_score = np.mean(scores)
            print(f"Layers: {layers}, Epochs: {epochs}, Batch Size: {batch_size}, Accuracy: {avg_score:.4f}")
            if avg_score > best_score:
                best_score = avg_score
                best_params = {'hidden_layers': layers, 'epochs': epochs, 'batch_size': batch_size}

print(f"Best Hyperparameters: {best_params}")

# Train the Model with Best Hyperparameters on Entire Dataset
best_model = build_model(hidden_layers=best_params['hidden_layers'])
history = best_model.fit(X, y_onehot, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=1)

# Plot Accuracy and Loss Graphs
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Cross-Validation Predictions
y_pred = cross_val_predict(
    estimator=tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=lambda: build_model(hidden_layers=best_params['hidden_layers']),
                                                             epochs=best_params['epochs'],
                                                             batch_size=best_params['batch_size'], verbose=0),
    X=X, y=y, cv=10
)

# Classification Report as Pandas DataFrame
report = classification_report(y, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
print(report_df)

# Confusion Matrix Heatmap
cm = confusion_matrix(y, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.unique(y), yticklabels=np.unique(y))
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()